### Sequence-to-sequence example

This notebook modifies a notebook from Challot's [Keras blog](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html). The code was slightly modified to correct for certain errors that were thrown using a newer version of tensorflow. 

The data used is from a dataset of English/French pairs of sentences, which you can download here: http://www.manythings.org/anki/

The link above has data sets of bilingual sentence pairs for dozens of languages. 

The approach below separates the encoding from the decoding, in part because training time is quite lengthy. 

One interesting feature of the approach below is that it generates the text character by character instead of word by word. 

### Read the data

Before getting started, let's look at the data. 

The code block below reads in the data, which looks like this:

```
Go.	Va !	CC-BY 2.0 (France) Attribution: . . .
```
The file is tab-delimited. The first item is an English sentence, the second the French translation, and the rest is information we don't care about. 

For each line in the file, the input text and target text are added to their own lists, while also gathering a set of characters used in the input language and a set of characters in the target language. 

In [2]:
data_path = 'data/fra-eng/fra.txt'
num_samples = 10000  # Number of samples to train on.

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

Next, we set up some parameters for encoding length based on the input data. 

In [3]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


### Vectorize the data

In order to vectorize the data, an index dictionary is created for the input language, and one for the target language. 

Then sparse matrices are created for:
* decoder input
* decoder target
* encoder input

In [5]:
import numpy as np

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [9]:
# look at the index dictionaries
print(input_token_index['a'])
print(target_token_index['a'])

44
45


In [10]:
# look at one of the sparse matrices

decoder_input_data[0]

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

### Set up encoder

In [18]:
from tensorflow.keras import layers, models

In [12]:
# some parameters

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

The next code block defines the input layer and the LSTM layer for the encoder. 

In [19]:
# Define an input sequence and process it.
encoder_inputs = layers.Input(shape=(None, num_encoder_tokens))
encoder = layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

### Set up decoder

In [20]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = layers.Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = layers.Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

### Set up the model

In [22]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

Next we train the model. This took about 22 minutes on a Mac mini. 

In [23]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Epoch 1/100
125/125 [==============================] - 14s 111ms/step - loss: 1.1914 - accuracy: 0.7242 - val_loss: 1.0451 - val_accuracy: 0.7078
Epoch 2/100
125/125 [==============================] - 13s 106ms/step - loss: 0.8613 - accuracy: 0.7684 - val_loss: 0.8535 - val_accuracy: 0.7676
Epoch 3/100
125/125 [==============================] - 13s 104ms/step - loss: 0.6980 - accuracy: 0.8051 - val_loss: 0.7219 - val_accuracy: 0.7902
Epoch 4/100
125/125 [==============================] - 13s 105ms/step - loss: 0.6000 - accuracy: 0.8263 - val_loss: 0.6504 - val_accuracy: 0.8076
Epoch 5/100
125/125 [==============================] - 14s 112ms/step - loss: 0.5487 - accuracy: 0.8400 - val_loss: 0.6161 - val_accuracy: 0.8179
Epoch 6/100
125/125 [==============================] - 13s 108ms/step - loss: 0.5097 - accuracy: 0.8503 - val_loss: 0.5975 - val_accuracy: 0.8241
Epoch 7/100
125/125 [==============================] - 13s 104ms/step - loss: 0.4800 - accuracy: 0.8584 - val_loss: 0.5550 -

### Inference mode

Now that the model is trained, here are the next steps:

* encode the input and retrieve initial decoder state
* run one step of decoder with this initial state and a 'start' token as target; the output will be the next token
* repeate with current target and current states

In [24]:
# Define sampling models
encoder_model = models.Model(encoder_inputs, encoder_states)

In [26]:
decoder_state_input_h = layers.Input(shape=(latent_dim,))
decoder_state_input_c = layers.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = models.Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

The following is a function to encode the input.

In [27]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

The rest of the code in the notebook takes in the encoder input sequence from the training data, and decoding it to get an output sentence. This is simply illustrating what the model learned, not evaluating how well it could perform on unseen data.

In [38]:
inputs = []
outputs = []

for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    
    inputs.append(input_texts[seq_index])
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    outputs.append(decode_sequence(input_seq))

In [40]:
# output a few

for i in range(10):
    print('-')
    print('Input sentence:', inputs[i])
    print('Decoded sentence:', outputs[i])


-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Who?
Decoded sentence: Qui ?

-
Input sentence: Wow!
Decoded sentence: Ça alors !

-
Input sentence: Fire!
Decoded sentence: Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide !

-
Input sentence: Jump.
Decoded sentence: Saute.

